# XAI FOR YOUTUBE ENGAGEMENT

StudentID|Full Name
-|-
21127050|Tran Nguyen Huan 
21127240|Nguyen Phat Dat

## Part 1: Crawl necessary data `Youtube` from file `Entube.csv`

## Import necessary libraries

In [1]:
import pandas as pd
import requests
import yt_dlp

### Crawl features `Title`, `Tags`, and `Thumbnail`

In [2]:
API_KEY = "*"
YOUTUBE_API_URL = "https://www.googleapis.com/youtube/v3/videos"

In [3]:
def get_video_metadata(video_id):
    try:
        params = {
            "part": "snippet",
            "id": video_id,
            "key": API_KEY
        }
        response = requests.get(YOUTUBE_API_URL, params=params)
        data = response.json()

        if "items" in data and len(data["items"]) > 0:
            snippet = data["items"][0]["snippet"]
            return {
                "Title": snippet["title"],
                "Tags": snippet.get("tags", []),
                "Thumbnail": snippet["thumbnails"]["high"]["url"],
            }
        else:
            print(f"No data found for video ID: {video_id}")
            return {"Title": None, "Tags": None, "Thumbnail": None}

    except Exception as e:
        print(f"Error processing video ID {video_id}: {e}")
        return {"Title": None, "Tags": None, "Thumbnail": None}

In [4]:
# df = pd.read_csv("data/EnTube.csv")

# video_data = [get_video_metadata(row["video_id"]) for _, row in df.iterrows()]

# crawled_df = pd.DataFrame(video_data)
# result_df = pd.concat([df, crawled_df], axis=1)
# result_df.to_csv("entube_with_metadata.csv", index=False)
# print("Crawling completed and saved to entube_with_metadata.csv")

In [5]:
df = pd.read_csv("data/EnTube.csv")

## Crawl feature `audio`

In [ ]:
import os
from yt_dlp import YoutubeDL

def download_audios(youtube_urls, output_path='data/audio/'):
    """
    Download and extract audio for a list of YouTube URLs using yt-dlp.

    Args:
        youtube_urls (list): List of YouTube video URLs to download.
        output_path (str): Directory where audio files will be saved. Defaults to 'data/audio/'.
    """
    if not youtube_urls:
        print("No URLs provided for download.")
        return

    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)

    # yt-dlp options for audio extraction
    ydl_opts = {
        'outtmpl': os.path.join(output_path, '%(id)s.%(ext)s'),  # Save with video ID as filename
        'format': 'bestaudio/best',  # Best quality audio
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,  # Suppress yt-dlp logs
        'ignoreerrors': True,  # Skip videos with errors
        'retries': 10,  # Retry failed downloads
        'fragment-retries': 10,  # Retry failed fragments
        'nooverwrites': True,
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download(youtube_urls)  # Pass the list of URLs for batch download
        print(f"Batch download completed for {len(youtube_urls)} videos.")
    except Exception as e:
        print(f"Failed to complete batch download: {e}")


def download_audios_from_csv(dataset, output_path='data/audio/'):
    """
    Download audio for all YouTube videos listed in a CSV dataset.

    Args:
        dataset (pd.DataFrame): DataFrame containing video links.
        output_path (str): Directory where audio files will be saved. Defaults to 'data/audio/'.
    """
    youtube_urls = dataset['video_link'].dropna().tolist()  # Extract non-null video links
    if youtube_urls:
        print(f"Starting batch download for {len(youtube_urls)} videos.")
        download_audios(youtube_urls, output_path)
    else:
        print("No valid video links found in the dataset.")

In [7]:
# # Download audio for all videos in the dataset
# download_audios_from_csv(df, output_path='/mnt/d/Thesis/Data/Audio')

## Crawl feature `video content`

In [8]:
import os
from yt_dlp import YoutubeDL

def download_video_only(youtube_urls, output_path='data/videos/'):
    """
    Download YouTube videos at 480p resolution without audio.

    Args:
        youtube_urls (list): List of YouTube video URLs to download.
        output_path (str): Directory where the videos will be saved. Defaults to 'data/videos/'.
    """
    if not youtube_urls:
        print("No URLs provided for download.")
        return

    # Ensure the output directory exists
    os.makedirs(output_path, exist_ok=True)

    # yt-dlp options for video-only download
    ydl_opts = {
        'outtmpl': os.path.join(output_path, '%(id)s.%(ext)s'),  # Save with video ID as filename
        'format': 'bestvideo[height<=480]',  # Video-only, capped at 480p
        'quiet': True,  # Suppress yt-dlp logs
        'ignoreerrors': True,  # Skip videos with errors
        'retries': 10,  # Retry failed downloads
        'fragment-retries': 10,  # Retry failed fragments
        'nooverwrites': True,
    }

    try:
        with YoutubeDL(ydl_opts) as ydl:
            ydl.download(youtube_urls)  # Pass the list of URLs for batch download
        print(f"Batch download completed for {len(youtube_urls)} videos.")
    except Exception as e:
        print(f"Failed to complete batch download: {e}")


def download_videos_from_csv(dataset, output_path='data/videos/'):
    """
    Download video-only files (no audio) for all YouTube links listed in a CSV dataset.

    Args:
        dataset (pd.DataFrame): DataFrame containing video links.
        output_path (str): Directory where videos will be saved. Defaults to 'data/videos/'.
    """
    youtube_urls = dataset['video_link'].dropna().tolist()  # Extract non-null video links
    if youtube_urls:
        print(f"Starting batch download for {len(youtube_urls)} videos.")
        download_video_only(youtube_urls, output_path)
    else:
        print("No valid video links found in the dataset.")


In [ ]:
download_videos_from_csv(df, output_path='/mnt/d/Thesis/Data/Video')

Starting batch download for 23738 videos.


ERROR: [youtube] D00vn3X7oI8: Private video. Sign in if you've been granted access to this video
